task name: NormalizeMinMaxMulticolumn
task description: Implement the NormalizeMinMaxMulticolumn example using ML.NET

In [ ]:
#r "nuget:Microsoft.ML,1.5.2"

In [ ]:
using System;
using System.Collections.Generic;
using System.Collections.Immutable;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using static Microsoft.ML.Transforms.NormalizingTransformer;

In [ ]:
var mlContext = new MLContext();

In [ ]:
public class DataPoint
{
    [VectorType(4)]
    public float[] Features { get; set; }

    [VectorType(3)]
    public float[] Features2 { get; set; }
}

var samples = new List<DataPoint>()
{
    new DataPoint()
    {
        Features = new float[4] { 1, 1, 3, 0 },
        Features2 = new float[3] { 1, 2, 3 }
    },
    new DataPoint()
    {
        Features = new float[4] { 2, 2, 2, 0 },
        Features2 = new float[3] { 3, 4, 5 }
    },
    new DataPoint()
    {
        Features = new float[4] { 0, 0, 1, 0 },
        Features2 = new float[3] { 6, 7, 8 }
    },
    new DataPoint()
    {
        Features = new float[4] {-1,-1,-1, 1 },
        Features2 = new float[3] { 9, 0, 4 }
    }
};

In [ ]:
var data = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var columnPair = new[]
{
    new InputOutputColumnPair("Features"),
    new InputOutputColumnPair("Features2")
};

var normalize = mlContext.Transforms.NormalizeMinMax(columnPair,
    fixZero: false);

var normalizeFixZero = mlContext.Transforms.NormalizeMinMax(columnPair,
    fixZero: true);

In [ ]:
var normalizeTransform = normalize.Fit(data);
var transformedData = normalizeTransform.Transform(data);
var normalizeFixZeroTransform = normalizeFixZero.Fit(data);
var fixZeroData = normalizeFixZeroTransform.Transform(data);

In [ ]:
var column = transformedData.GetColumn<float[]>("Features").ToArray();
var column2 = transformedData.GetColumn<float[]>("Features2").ToArray();

for (int i = 0; i < column.Length; i++)
    Console.WriteLine(string.Join(", ", column[i].Select(x => x
    .ToString("f4"))) + "\t\t" +
    string.Join(", ", column2[i].Select(x => x.ToString("f4"))));

var columnFixZero = fixZeroData.GetColumn<float[]>("Features").ToArray();
var column2FixZero = fixZeroData.GetColumn<float[]>("Features2").ToArray();

Console.WriteLine(Environment.NewLine);

for (int i = 0; i < column.Length; i++)
    Console.WriteLine(string.Join(", ", columnFixZero[i].Select(x => x
    .ToString("f4"))) + "\t\t" +
    string.Join(", ", column2FixZero[i].Select(x => x.ToString("f4"))));

var transformParams = normalizeTransform.GetNormalizerModelParameters(0)
    as AffineNormalizerModelParameters<ImmutableArray<float>>;

var transformParams2 = normalizeTransform.GetNormalizerModelParameters(1)
    as AffineNormalizerModelParameters<ImmutableArray<float>>;

Console.WriteLine(Environment.NewLine);

Console.WriteLine($"The 1-index value in resulting array would be " +
    $"produced by:");

Console.WriteLine(" y = (x - (" + (transformParams.Offset.Length == 0 ?
    0 : transformParams.Offset[1]) + ")) * " + transformParams
    .Scale[1]);